#### データソースを準備

In [ ]:
from trafilatura import fetch_url, extract

# 遊戯王カードwikiのページを取得
url = 'https://yugioh-wiki.net/index.php?%A1%DA%A5%A2%A5%EB%A5%D0%A5%B9%A4%CE%CD%EE%B0%FD%A1%DB'
filename = 'deck_albaz.txt'
document = fetch_url(url)

# extract()でテキストを抽出
text = extract(document)

# テキストをファイルに保存
if text is not None:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)
else:
    print("No text could be extracted from the document.")

#### チャンク分割

In [ ]:
from langchain.document_loaders import TextLoader

# テキストファイルを読み込む
loader = TextLoader(filename, encoding="utf-8")
raw_docs = loader.load()

from langchain.text_splitter import CharacterTextSplitter

# チャンク分割を定義
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0,
)

# 読み込んだテキストのチャンク分割を実行
docs = text_splitter.split_documents(raw_docs)

print(len(docs))

#### embedding

In [ ]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()

# OpenAIのEmbeddingモデルを定義
embeddings = OpenAIEmbeddings(deployment=os.environ["DEPLOYMENT_NAME_EMBEDDINGS"])

from langchain.vectorstores import Chroma

# Chromaベクトルストアを定義
db = Chroma.from_documents(docs, embeddings)

#### 回答

In [ ]:
from langchain.chains import  RetrievalQA
from langchain.chat_models import AzureChatOpenAI

retriever = db.as_retriever()

# チャットの組み立て
chat = AzureChatOpenAI(
    deployment_name=os.environ["DEPLOYMENT_NAME"],
    temperature=0
)

# RAGのためのchainを組み立て
rag_qa = RetrievalQA.from_chain_type(
    llm = chat,
    chain_type= "stuff",
    retriever = retriever,
    return_source_documents = True,
)

# 質問文のリストを作成
query_list = ["「アルバスの落胤」デッキのキーカードは？",
              "「アルバスの落胤」の攻は？",
              "「アルバスの落胤」デッキの弱点は？",
              "烙印融合のテキストを教えて",
              "「アルバスの落胤」デッキの派生デッキを3つ教えて",
              "「アルバスの落胤」デッキではリンクモンスターは何が採用される？",
              "「アルバスの落胤」デッキでよく使われるカウンター罠は？",
              "「アルバスの落胤」デッキの戦術は？",
              "「烙印融合」をサーチする方法を教えて",
              "「アルバスの落胤」デッキでは召喚権をどのモンスターに使う？",
              "「アルバスの落胤」デッキにティアラメンツを混ぜるメリットは？",
]

# LLMに渡すチャンクを格納する空リストを作成
context_list = []

# LLMからの回答を格納する空リストを作成
result_list = []

In [ ]:
# rag_qaを回して、context_listとresult_listに選ばれたチャンクと回答を格納していく
for query in query_list:
    rag_result = rag_qa({"query": query})

    # contextを取得して、page_contentフィールドをcontext_listに追加に格納
    context = rag_result["source_documents"][0]
    context_list.append([context.page_content])

    # resultを取得して、result_listに格納
    result_list.append(rag_result["result"])

print("チャンクを表示")
print(*context_list, sep='\n++++++++++++\n')
print("*************")
print("回答を表示")
print(*result_list, sep='\n++++++++++++\n')

#### ragasをAzure OpenAIで使用する

In [ ]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    harmfulness,
]

In [ ]:
import os
from dotenv import load_dotenv

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from ragas.llms import LangchainLLM

load_dotenv()

# Azure OpenAI Chatモデルの組み立て
azure_model = AzureChatOpenAI(
    deployment_name=os.environ['DEPLOYMENT_NAME'],
    model=os.environ['MODEL_NAME'],
    openai_api_base=os.environ['OPENAI_API_BASE'],
    openai_api_type=os.environ['OPENAI_API_TYPE'],  # 'azure'
)

# azure_modelのインスタンスをラップする
ragas_azure_model = LangchainLLM(azure_model)

answer_relevancy.llm = ragas_azure_model

# Embeddingモデルの組み立て
azure_embeddings = OpenAIEmbeddings(
    deployment=os.environ['DEPLOYMENT_NAME_EMBEDDINGS'],
    model=os.environ['MODEL_NAME_EMBEDDINGS'],
    openai_api_base=os.environ['OPENAI_API_BASE'],
    openai_api_type=os.environ['OPENAI_API_TYPE'], 
)

answer_relevancy.embeddings = azure_embeddings

In [ ]:
for m in metrics:
    m.__setattr__('llm', ragas_azure_model)
    m.__setattr__('embeddings', azure_embeddings)

In [ ]:
# 真の回答をリストに格納
truth_list = [['アルバスの落胤'],
              ['攻撃力は1800？'],
              ['エクストラデッキメタ'],
              ['このカード名のカードは１ターンに１枚しか発動できず、このカードを発動するターン、自分は融合モンスターしかＥＸデッキから特殊召喚できない。(1)：自分の手札・デッキ・フィールドから、融合モンスターカードによって決められた融合素材モンスター２体を墓地へ送り、「アルバスの落胤」を融合素材とするその融合モンスター１体をＥＸデッキから融合召喚する。'],
              ['ドラグマ軸、トライブリゲード軸、スプリガンズ軸'],
              ['ストライカー・ドラゴン、天球の聖刻印、クロシープ'],
              ['特にありません'],
              ['アルバスの落胤や超融合で相手モンスターを除去してボードアドバンテージを稼ぐ'],
              ['デスピアの導化アルベルの召喚・特殊召喚成功時効果でサーチする'],
              ['アルバスの落胤'],
              ['元の記事には記載がないので不明'],
]

In [ ]:
from datasets import Dataset

# データセットの組み立て
ds = Dataset.from_dict(
    {
        'question': query_list,
        'answer': result_list,
        'contexts': context_list,
        'ground_truths': truth_list,
    }
)

In [ ]:
from ragas import evaluate

result = evaluate(
    ds,
    metrics=metrics,
)

result

In [ ]:
df = result.to_pandas()
df.head()